In [1]:
import numpy as np
import pynq
from pynq import Overlay
import time

overlay = Overlay("/home/xilinx/jupyter_notebooks/hdc_test/hdc_v2.4.bit")

encode_inst = overlay.encode_0

num_dims = 4096
num_features = 784


In [2]:
array_sample  = pynq.allocate(shape=(num_features,), dtype=np.float32)
array_basis   = pynq.allocate(shape=(num_dims*num_features,), dtype=np.float32)
array_encoded = pynq.allocate(shape=(num_dims,), dtype=np.float32)

array_sample[:]  = np.random.random(size=num_features)
array_basis[:]   = np.random.random(size=num_dims*num_features)
array_encoded[:] = np.zeros(shape=(num_dims,), dtype=np.float32)

array_sample.sync_to_device()
array_basis.sync_to_device()
array_encoded.sync_to_device()

array_basis.physical_address
type(array_basis.physical_address)

print(f"array_encoded = {array_encoded}")

array_encoded = [0. 0. 0. ... 0. 0. 0.]


In [3]:
# help(vecadd_inst.mmio)
encode_inst.mmio.write_reg(0x10, array_sample.physical_address)
encode_inst.mmio.write_reg(0x14, 0)
encode_inst.mmio.write_reg(0x1c, array_basis.physical_address)
encode_inst.mmio.write_reg(0x20, 0)
encode_inst.mmio.write_reg(0x28, array_encoded.physical_address)
encode_inst.mmio.write_reg(0x2c, 0)

In [4]:

ip_status = encode_inst.read(0x00)
print(ip_status)


ip_status = encode_inst.read(0x00)
print(ip_status)

encode_inst.write(0x00, 1)
ip_status = encode_inst.read(0x00)
print(ip_status)
while (ip_status == 4):
    ip_status = encode_inst.read(0x00)

ip_status = encode_inst.read(0x00)
print(ip_status)
print('accelerator done')

4
4
1
1
accelerator done


In [5]:
ip_status = encode_inst.read(0x00)
print(ip_status)

array_sample.sync_from_device()
array_basis.sync_from_device()
array_encoded.sync_from_device()

print(f"array_sample = {array_sample}")
print(f"array_basis = {array_basis}")
print(f"array_encoded = {array_encoded}")

basis_2d = np.reshape(array_basis, (num_features, num_dims), order='F')

print(f"basis_2d = {basis_2d}")


ref_encoded = np.dot(array_sample, basis_2d)
encoded_max = np.max(ref_encoded)
encoded_min = np.min(ref_encoded)
ref_encoded = -1.0 + (2.0 / (encoded_max - encoded_min)) * (ref_encoded - encoded_min);

print(f"reference = {ref_encoded}")

np.count_nonzero(array_encoded > 1e-05 )


14
array_sample = [3.28846201e-02 9.34779048e-01 3.74145597e-01 5.01851559e-01
 1.34413421e-01 9.51684475e-01 3.21220756e-01 6.82641983e-01
 3.78489047e-01 6.58541560e-01 8.33991170e-01 8.73657882e-01
 2.58508652e-01 4.28341597e-01 7.57932365e-01 4.01088655e-01
 5.47323644e-01 6.49912775e-01 9.13021624e-01 1.57562360e-01
 4.62992638e-02 7.30574429e-01 1.40550226e-01 9.76714611e-01
 2.01780289e-01 6.99677646e-01 3.86799574e-02 2.57850215e-02
 5.09354413e-01 1.77320525e-01 8.74655545e-01 7.40606248e-01
 6.20221913e-01 1.81591198e-01 5.34195721e-01 9.15544510e-01
 4.77849215e-01 6.12668216e-01 1.40236348e-01 9.51864481e-01
 5.68984032e-01 6.11545324e-01 9.05274153e-01 4.54347581e-01
 3.73771906e-01 9.43885148e-01 3.25301200e-01 2.10051879e-01
 9.69181120e-01 8.14365208e-01 5.36318064e-01 2.63941467e-01
 9.82701123e-01 7.99996376e-01 7.92665482e-01 6.13353133e-01
 3.50050002e-01 4.62609306e-02 8.35751891e-01 9.46727395e-02
 8.38595554e-02 7.54093289e-01 9.69530106e-01 8.88917983e-01
 6.183

reference = [ 0.54719174 -0.5067171   0.15572667 ...  0.15168583 -0.08097672
 -0.33272415]


2006

In [6]:
a = [1, 0]
b = [[4, 1], [2, 2]]
np.dot(a, b)

array([4, 1])